# Jupyter Notebook: Einfaches CNN und AlexNet Implementierung mit CIFAR-10

In diesem Notebook lernst du, wie du ein einfaches Convolutional Neural Network (CNN) auf dem CIFAR-10-Datensatz trainierst. Du wirst die Trainings- und Validierungsverluste nach jeder Epoche plotten und eine Heatmap-Funktion auf ein Beispielbild aus dem Validierungsdatensatz anwenden, um aktivierte Bildregionen zu visualisieren. Im zweiten Teil wirst du angeleitet, AlexNet anhand eines Bildes der Architektur selbst zu implementieren.

## 1. Datensatz vorbereiten

Du wirst den CIFAR-10-Datensatz verwenden, der 60.000 32x32-Farbbilder in 10 Klassen enthält. Der Trainingsdatensatz wird in 80% Training und 20% Validierung aufgeteilt.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable

# Transformationen für die Bilder
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Trainingsdatensatz laden
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# In Trainings- und Validierungssatz aufteilen (80% Training, 20% Validierung)
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_subset, val_subset = random_split(trainset, [train_size, val_size])

trainloader = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=2)

# Testdatensatz laden
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Klassen
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## 2. Einfaches CNN definieren

Definiere ein einfaches CNN mit zwei Convolutional Layers und drei Fully Connected Layers.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Modell initialisieren und auf GPU verschieben, falls verfügbar
device = torch.device("cuda" if torch.cuda.is_available() else "mps")
model = SimpleCNN().to(device)

## 3. Training des Modells

Trainiere das Modell mit dem Adam-Optimizer und der Cross-Entropy Loss-Funktion. Nach dem Training werden die Verluste geplottet.

In [ ]:
import torch.optim as optim

# Verlustfunktion und Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss', marker='o')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Val Loss', marker='s')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Trainings- und Validierungsverluste')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Trainingsschleife
num_epochs = 10
train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(trainloader)
    train_losses.append(train_loss)

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss /= len(valloader)
    val_losses.append(val_loss)
    val_accuracy = correct / total
    val_accuracies.append(val_accuracy)


    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

plot_losses(train_losses, val_losses)

## 4. Evaluation auf Testdatensatz

Evaluiere das Modell auf dem Testdatensatz, um die endgültige Leistung zu überprüfen.

In [ ]:
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_loss /= len(testloader)
accuracy = correct / total

print(f'Test Loss: {val_loss:.4f}, Test Accuracy: {accuracy:.4f}')


## 5. AlexNet implementieren

In diesem Abschnitt wirst du AlexNet anhand eines Bildes der Architektur implementieren. Deine Aufgabe ist es, die Architektur in PyTorch zu programmieren, indem du die Schichten und Verbindungen aus dem Bild nachbaust.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ad/AlexNet_block_diagram.svg/1920px-AlexNet_block_diagram.svg.png)




### Anleitung zur Implementierung

Um AlexNet zu implementieren, folge diesen Schritten:
**1. Analysiere das Bild**:
_ Schaue dir die Schichten der Architektur genau an. Beachte die Anzahl der Convolutional Layers, Pooling Layers und Fully Connected Layers.
- Identifiziere die Anzahl der Filter, Kernel-Größen, Strides, Padding-Werte und Aktivierungsfunktionen (z.B. ReLU).
- Achte auf Dropout-Schichten und die Größe der Fully Connected Layers.

**2. Erstelle das Modell in PyTorch**:
- Definiere eine Klasse AlexNet, die von nn.Module erbt
- Implementiere die Convolutional Layers mit nn.Conv2d, Pooling Layers mit nn.MaxPool2d und Fully Connected Layers mit nn.Linear.
- Füge ReLU-Aktivierungen (F.relu oder nn.ReLU) und Dropout (nn.Dropout) hinzu, wie im Bild angegeben.
- Stelle sicher, dass die Eingabegröße (32x32 für CIFAR-10) mit der Architektur kompatibel ist. Du musst möglicherweise Kernel-Größen oder Strides anpassen, da AlexNet ursprünglich für 224x224-Bilder entwickelt wurde.

**3. Forward-Pass**:
- Implementiere die forward-Methode, die die Eingabe durch die Schichten führt.
- Achte darauf, die Dimensionen nach den Convolutional Layers zu flatten (z.B. mit x.view), bevor du die Fully Connected Layers anwendest.

**4. Training**:
- Trainiere dein AlexNet-Modell ähnlich wie das einfache CNN oben. Verwende den gleichen trainloader, valloader und testloader, die Adam-Optimizer und Cross-Entropy Loss.
- Du kannst die gleichen Funktionen (validate_model, plot_losses, create_heatmap) wiederverwenden, um Verluste zu plotten und Heatmaps zu erstellen.

### Tipps
- **Dimensionen überprüfen**: Berechne die Ausgabegrößen jeder Schicht, um sicherzustellen, dass die Fully Connected Layers die richtige Eingabegröße erhalten. Für CIFAR-10 (32x32) musst du die Parameter anpassen, da AlexNet für größere Bilder ausgelegt ist.
- **Debugging**: Teste dein Modell mit einem einzelnen Batch, um sicherzustellen, dass die Dimensionen korrekt sind (z.B. output = model(torch.randn(1, 3, 32, 32).to(device))).
- **Anpassungen**: Falls die ursprüngliche AlexNet-Architektur zu komplex ist, kannst du die Anzahl der Filter oder Schichten reduzieren, um sie für CIFAR-10 zu optimieren.

### Beispiel für den Anfang

Hier ist ein Gerüst, um dir den Einstieg zu erleichtern, ohne die vollständige Implementierung zu verraten:

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            # Conv1: 3 input channels (RGB), 64 filters, kernel 3x3, stride 1, padding 1
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 64x16x16

            # Conv2: 64 input channels, 192 filters, kernel 3x3, padding 1
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 192x8x8

            # Conv3: 192 input channels, 384 filters, kernel 3x3, padding 1
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # Conv4: 384 input channels, 256 filters, kernel 3x3, padding 1
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # Conv5: 256 input channels, 256 filters, kernel 3x3, padding 1
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 256x4x4
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 4 * 4, 1024),  # Angepasst an 4x4 Feature Maps
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 4 * 4)
        x = self.classifier(x)
        return x

# Modell initialisieren und auf GPU verschieben
alexnet = AlexNet().to(device)

Aufgabe: Fülle die features und classifier Abschnitte basierend auf dem Bild der Architektur aus. Trainiere das Modell anschließend mit der gleichen Trainingsschleife wie beim einfachen CNN und überprüfe die Leistung auf dem Testdatensatz.

### 5.2 Erklärung der Anpassungen

Die Original-AlexNet-Architektur (für 224x224-Bilder) wurde wie folgt an CIFAR-10 (32x32-Bilder) angepasst:

- **Kernel-Größen und Strides**: Kleinere Kernel (3x3 statt 11x11) und Strides (1 statt 4) für Conv1, um die kleineren Eingabebilder zu berücksichtigen.
- **Pooling**: Max-Pooling-Schichten wurden beibehalten, aber mit kleineren Kerneln (2x2 statt 3x3), um die Feature Maps schrittweise zu reduzieren.
- **Fully Connected Layers**: Die Anzahl der Neuronen wurde auf 1024 reduziert (statt 4096), und die Eingabegröße der ersten FC-Schicht wurde an die Ausgabe der letzten Convolutional Layer (256x4x4) angepasst.
- **Dropout**: Dropout (p=0.5) wurde beibehalten, um Overfitting zu verhindern.
- **Dimensionen**: Die Architektur wurde so gestaltet, dass die Feature Maps von 32x32 bis 4x4 reduziert werden, kompatibel mit CIFAR-10.

In [ ]:
# Optimizer für AlexNet
optimizer_alexnet = optim.Adam(alexnet.parameters(), lr=0.001)

# Trainingsschleife für AlexNet
train_losses_alexnet = []
val_losses_alexnet = []
val_accuracies_alexnet = []

for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer_alexnet.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_alexnet.step()

        running_loss += loss.item()

    train_loss = running_loss / len(trainloader)
    train_losses_alexnet.append(train_loss)

    alexnet.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = alexnet(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss /= len(valloader)
    val_losses_alexnet.append(val_loss)
    val_accuracy = correct / total

    val_accuracies_alexnet.append(val_accuracy)


    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

plot_losses(train_losses_alexnet, val_losses_alexnet)

In [ ]:
alexnet.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_loss /= len(testloader)
accuracy = correct / total

print(f'Test Loss: {val_loss:.4f}, Test Accuracy: {accuracy:.4f}')
